# LLM Classifiers

**Goal:** Given a sentence as input, classify it as either a prediction or non-prediction.

In [1]:
import os
import sys

import pandas as pd

from tqdm import tqdm

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

# import log_files
from data_processing import DataProcessing
from classification_models import EvaluationMetric
from text_generation_models import TextGenerationModelFactory

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Load Data

In [3]:
print("======= LOAD DATA =======")

======= LOAD DATA =======


In [4]:
base_data_path = os.path.join(notebook_dir, '../data/')
combine_data_path = os.path.join(base_data_path, 'combined_generated_fin_phrase_bank')
X_test_set_path = os.path.join(combine_data_path, 'x_test_set-v1.csv')
y_test_set_path = os.path.join(combine_data_path, 'y_test_set-v1.csv')

In [5]:
X_test_df = DataProcessing.load_from_file(X_test_set_path, 'csv')
X_test_df.drop(columns=['Unnamed: 0'], inplace=True)
X_test_df.head(7)

,Base Sentence,Sentence Label,Embedding,Normalized Embeddings
0,"Clothing retail chain Sepp+Æl+Æ 's sales increased by 8 % to EUR 155.2 mn , and operating profit rose to EUR 31.1 mn from EUR 17.1 mn in 2004 .",0,[-1.81379244e-01 3.26961249e-01 1.23493910e-01 -6.83715194e-02\n 2.05705479e-01 -1.32566988e-01 6.46815300e-02 4.80639301e-02\n 1.11243963e-01 1.16633594e+00 -3.26421767e-01 4.92487364e-02\n -4.69185673e-02 7.74623454e-03 1.52571976e-01 -6.91895038e-02\n 2.16319803e-02 1.07760060e+00 -1.00143224e-01 -9.74733476e-03\n 3.12185083e-02 8.63965750e-02 -5.83441146e-02 -1.37795210e-01\n 1.45358860e-01 7.57031664e-02 -1.95762575e-01 -4.27063033e-02\n -4.27523181e-02 1.41373962e-01 -1.76752619e-02 -6.47352412e-02\n -7.01081008e-02 4.31978516e-02 4.62445021e-02 -1.18927144e-01\n -6.29041046e-02 -6.16270676e-02 7.53541365e-02 1.11510776e-01\n -4.29402962e-02 -5.15806749e-02 2.08475545e-01 -7.44214728e-02\n -3.59057635e-02 5.50432652e-02 4.90117073e-02 2.19748504e-02\n 5....,[-8.70693088e-01 9.29072917e-01 1.81525087e+00 -8.27133536e-01\n 2.25419474e+00 -1.47255635e+00 1.30219591e+00 6.11075044e-01\n 8.57213974e-01 -2.04687071e+00 -3.36014867e-01 2.54737914e-01\n -1.41734147e+00 1.72387421e-01 2.00326610e+00 -6.32218063e-01\n 7.39459515e-01 -1.71444774e-01 2.27673307e-01 2.56912142e-01\n 6.52836114e-02 3.95675719e-01 -9.40405071e-01 -6.87238336e-01\n 1.60017776e+00 8.45497325e-02 -1.07574511e+00 -1.15273201e+00\n -1.10858238e+00 8.71268451e-01 -1.68091118e-01 -1.22336841e+00\n -1.05818677e+00 -4.21162724e-01 -1.83520094e-01 -1.66415644e+00\n -4.83112693e-01 -1.77971220e+00 1.43746281e+00 1.96571779e+00\n -8.22783291e-01 -1.56685221e+00 1.68052936e+00 -7.95643389e-01\n -1.04292214e+00 9.29304719e-01 1.52841580e+00 7.96010494e-01\n 2....
1,"On 2027/08/20, the Federal Reserve speculates that interest rates at major banks will likely increase.",1,[-1.81962401e-01 2.51789659e-01 -6.82267621e-02 -1.99264623e-02\n -8.90381262e-02 -6.99648187e-02 -1.15359366e-01 -4.32647616e-02\n -7.00312927e-02 2.39203525e+00 -2.53967375e-01 4.79923822e-02\n 1.87749177e-01 3.72870415e-02 -6.69317171e-02 -6.66592419e-02\n -1.47542655e-01 1.03909945e+00 -1.73058122e-01 -3.13351192e-02\n -1.90051142e-02 1.15966164e-01 -1.18214618e-02 -8.65336806e-02\n -1.89228803e-02 1.55890629e-01 -1.02456011e-01 5.62909395e-02\n -1.22471154e-03 5.37542142e-02 -3.22786272e-02 9.06065181e-02\n 1.37776569e-01 -3.36506404e-03 3.07458919e-02 5.67997433e-02\n -5.82847483e-02 1.10756606e-01 -4.76526432e-02 -3.98236886e-02\n -5.81108145e-02 3.09332926e-02 8.60598236e-02 -6.81905076e-02\n 1.33367687e-01 4.05994616e-02 -1.25284176e-02 -8.88468400e-02\n -2....,[-0.87786 -0.08358515 -1.2822452 -0.03210158 -1.6880763 -0.47773218\n -2.0795083 -0.57460356 -1.5074805 2.1458838 0.66962874 0.23881108\n 2.0131867 0.6769342 -0.2582024 -0.5840274 -2.3738577 -0.44169456\n -0.98855215 -0.10888498 -0.9732976 0.8344823 -0.17179996 0.04504178\n -0.9188829 1.1540416 0.3655062 0.52718747 -0.40584785 -0.39582273\n -0.43562493 1.2508281 2.2093909 -1.0789686 -0.49699044 1.3025148\n -0.4217364 0.93240243 -0.6820107 -0.30641684 -1.0963736 -0.26848567\n 0.00996405 -0.6897794 2.1643772 0.6851629 0.6662045 -0.56026655\n -0.91432357 1.1897789 0.43965554 -0.77815723 -0.19758885 2.0060427\n 0.28865618 -0.09320762 0.3796414 -2.1365964 -1.618398 1.0528172\n 1.4662083 0.41146848 0.81602204 3.0982656 -0.9026434...
2,"Consolidated net sales increased 16 % to reach EUR74 .8 m , while operating profit amounted to EUR0 .9 m compared to a loss of EUR0 .7 m in the prior year period .",0,[-1.54190257e-01 2.46810824e-01 -6.17762655e-02 7.55247250e-02\n -2.77188867e-02 -1.83458388e-01 9.04082379e-04 7.04187900e-02\n 1.32708490e-01 1.65489316e+00 -3.38965923e-01 1.12873256e-01\n -4.17059436e-02 1.10604092e-02 2.69969776e-02 -9.27351117e-02\n 1.27250105e-02 1.32193303e+00 -2.12390900e-01 7.88436830e-02\n 7.81772956e-02 6.12183416e-04 -7.47607872e-02 1.64486654e-02\n 1.09230407e-01 4.25606733e-03 -8.09814483e-02 5.87075278e-0

In [6]:
y_test_df = DataProcessing.load_from_file(y_test_set_path, 'csv')
y_test_df.drop(columns=['Unnamed: 0'], inplace=True)
# print(f"\t{y_test_df.head(7)}")

In [ ]:
print(f"\tShape: {X_test_df.shape}, \nSubset of Data:{X_test_df.head(3)}")
# df.shape, df.head(3)

print(f"\tShape: {y_test_df.shape}, \nSubset of Data:{y_test_df.head(3)}")
# df.shape, df.head(3)

In [ ]:
# for idx, row in X_test_df.iterrows():
#     text = row['Base Sentence']
#     label = row['Sentence Label']
#     embedding = row['Embedding']
#     print(type(embedding))
#     norm_embedding = row['Normalized Embeddings']
#     if idx < 7:
#         print(f"{idx}\n Sentence: {text}\n Label: {label}\n Embeddings Shape: {embedding}\n\t Embeddings Subset [:6]: {embedding[:6]} \n Norm Embeddings: {norm_embedding.shape}, \n\tNorm Embeddings Subset [:6]: {norm_embedding[:6]}")
# print(f"\tShape: {y_test_df.shape}, \nSubset of Data:{y_test_df.head(7)}")

<class 'str'>


AttributeError: 'str' object has no attribute 'shape'

## Load Prompt

In [ ]:
# prediction_properties = PredictionProperties.get_prediction_properties()
# prediction_requirements = PredictionProperties.get_requirements()
# system_identity_prompt = "You are an expert at identifying specific types of sentences by knowing the sentence format."
# prediction_examples_prompt = """Some examples of predictions in the PhraseBank dataset are
#     1. According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
#     2. According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales .
#     3. Its board of directors will propose a dividend of EUR0 .12 per share for 2010 , up from the EUR0 .08 per share paid in 2009 .
# """
# non_prediction_examples_prompt = """Some examples of non-predictions in the PPhraseBank dataset are
#     1. Net sales increased to EUR193 .3 m from EUR179 .9 m and pretax profit rose by 34.2 % to EUR43 .1 m. ( EUR1 = USD1 .4 )
#     2. Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales came from the acquisitions made in 2009 .
#     3. STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) raised the fair value for shares in four of the largest Nordic forestry groups .
# """
# # goal_prompt = "Given the above, identify the prediction."

# base_prompt = f"""{system_identity_prompt} The sentence format is based on: 
    
#     {prediction_properties}
#     Enforce: {prediction_requirements}
#     Know: {prediction_examples_prompt}
#     Know: {non_prediction_examples_prompt}

# """
# base_prompt

In [ ]:
prompt_1 = """ 

Role: 
You are a linguist expert. You are acting as a prediction detector. Your task is to identify if a given sentence is a prediction about the future.

Background:
A prediction is a statement about what someone thinks will happen in the future.
Examples of predictions:
- "It will rain tomorrow." (Yes)
- "The stock market is expected to rise next quarter." (Yes)
- "I am going to the store." (No)
- “Lakers will win the championship. ”(Yes)

A prediction may contain: source, target, date, outcome.
"""

## Models

In [ ]:
tgmf = TextGenerationModelFactory()

# Option 1: Specific models
# models = tgmf.create_instances(['llama-3.1-8b-instant', 'llama-3.3-70b-versatile'])

# Option 2: All Groq models
# models = tgmf.create_instances(tgmf.get_groq_model_names())

# Option 3: All NaviGator models
models = tgmf.create_instances(tgmf.get_navigator_model_names())

# Option 4: All available models
# models = tgmf.create_instances()

# Option 5: Mix and match
# custom_models = ['llama-3.1-70b-instruct', 'mistral-small-3.1', 'llama-3.1-8b-instant']
# models = tgmf.create_instances(custom_models)
models

In [ ]:
import json
import re

def parse_json_response(response):
    """Parse JSON response from LLM to extract label and reasoning"""
    try:
        # Extract JSON if there's extra text
        json_match = re.search(r'\{.*\}', response, re.DOTALL)
        if json_match:
            data = json.loads(json_match.group())
            return data.get('label'), data.get('reasoning')
    except Exception as e:
        print(f"Error parsing JSON: {e}")
        return None, None

In [ ]:
def llm_certifier(data: str, base_prompt: str, model):
    
        prompt = f""" Given this: {base_prompt}. Also given the sentence '{data}', your task is to analyze the sentence and determine if it is a prediction. If prediction, generate label as 1 and if non-prediction generate label as 0.
        Respond ONLY with valid JSON in this exact format:
        {{"label": 0, "reasoning": "your explanation here"}}
        Examples:
        - "It will rain tomorrow." → {{"label": 1, "reasoning": "Contains the future tense words 'will' and 'tomorrow'"}}
        - "The stock market is expected to rise next quarter." → {{"label": 1, "reasoning": "Contains future tense words 'is expected'"}}
        - "I am going to the store." → {{"label": 0, "reasoning": "Does not contain a future tense word"}}
        - "Lakers will win the championship." → {{"label": 1, "reasoning": "Contains the future tense word 'will'"}}
        """
        idx = 1
        if idx == 1:
            #   print(f"\tPrompt: {prompt}")
              idx = idx + 1
        input_prompt = model.user(prompt)
        raw_text_llm_generation = model.chat_completion([input_prompt])
        
        # Parse the JSON response
        label, reasoning = parse_json_response(raw_text_llm_generation)
        
        return raw_text_llm_generation, label, reasoning

In [ ]:
print("======= PROMPT + MODEL -> LABEL and REASONING =======")

In [ ]:
    # content : meta :: text : meta_data
results = []
for idx, row in X_test_df.iterrows():
    text = row['Base Sentence']
    print(f"{idx} --- Sentence: {text}")
    for model in models:
        # print(model.__name__())
        raw_response, llm_label, llm_reasoning = llm_certifier(text, prompt_1, model)
        print(f"\tModel: {model.__name__()}\n\t\tLabel:', {llm_label}\n\t\tReasoning:', {llm_reasoning}")
        result = (text, raw_response, llm_label, llm_reasoning, model.__name__())
        results.append(result)


In [ ]:
results

In [ ]:
# groupby text

In [ ]:
results_with_llm_label_df = pd.DataFrame(results, columns=['text', 'raw_response', 'llm_label', 'llm_reasoning', 'llm_name'])
results_with_llm_label_df

In [ ]:
y_test_df.rename(columns={'Sentence Label' : 'Actual Label'}, inplace=True)

In [ ]:
def get_llm_labels(df, model_name):
    filt_llama = (df['llm_name'] == model_name)
    filt_df = df[filt_llama]
    return filt_df['llm_label']


test_and_models_df = pd.concat([X_test_df.loc[:, :], y_test_df], axis=1)
# test_and_models_df.columns = ['Sentence', 'Actual Label']
for model in models:
    print(model.__name__())
    model_labels = get_llm_labels(results_with_llm_label_df, model.__name__())
    test_and_models_df[model.__name__()] = model_labels.to_numpy().ravel()
test_and_models_df

## Evaluation

In [ ]:
print("======= EVALUATION/RESULTS =======")

In [ ]:
get_metrics = EvaluationMetric()
get_metrics

In [ ]:
actual_label = test_and_models_df['Actual Label'].values
for ml_model in models:
    ml_model_name = ml_model.__name__()
    print(f"Actual Label:\t\t{actual_label}")
    ml_model_predictions = test_and_models_df[ml_model_name].values
    print(f"{ml_model_name}:\t\t{ml_model_predictions}")
    get_metrics.eval_classification_report(y_test_df, ml_model_predictions)